Copyright 2023 Recursion

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None

from efaar_benchmarking.data_loading import load_cpg16_crispr
from efaar_benchmarking.efaar import *
from efaar_benchmarking.benchmarking import benchmark
from efaar_benchmarking.plotting import plot_recall

recall_threshold_pairs = []
start = 0.01
end = 0.99
step = 0.01

while start <= .105 and end >= .895:
    recall_threshold_pairs.append((round(start,2), round(end,2)))
    start += step
    end -= step

print(recall_threshold_pairs)

In [ ]:
features, metadata = load_cpg16_crispr() # loading may take some time if the files are not cached yet, depending on the speed of your internet connection
embeddings = embed_align_by_pca(features.values)
embeddings = align_on_controls(embeddings, metadata)
map_data = aggregate(embeddings, metadata)
metrics = benchmark(map_data, recall_thr_pairs=recall_threshold_pairs, pert_label_col="Metadata_Symbol")
plot_recall(metrics)
metrics.groupby('source')['recall_0.05_0.95'].mean()